In [2]:
import os
import struct
import pandas as pd
import numpy as np
import time, datetime
import multiprocessing
import talib as tdx
from easyquant import MongoIo
import matplotlib.pyplot as plt
from easyquant.indicator.base import *

In [ ]:
def readTdxLdayFile(fname="sh000001.day"):
  dataSet=[]
  with open(fname,'rb') as fl:
    buffer=fl.read() #读取数据到缓存
    size=len(buffer) 
    rowSize=32 #通信达day数据，每32个字节一组数据
    code=os.path.basename(fname).replace('.day','')
    for i in range(0,size,rowSize): #步长为32遍历buffer
      row=list( struct.unpack('IIIIIfII',buffer[i:i+rowSize]) )
      row[1]=row[1]/100
      row[2]=row[2]/100
      row[3]=row[3]/100
      row[4]=row[4]/100
      row.pop() #移除最后无意义字段
      row.insert(0,code)
      dataSet.append(row) 
  data=pd.DataFrame(data=dataSet,columns=['code','tradeDate','open','high','low','close','amount','vol'])
  data['date'] = data['tradeDate'].apply(lambda x: datetime.datetime.strptime(str(x),'%Y%m%d').date())
  data=data.set_index(['date']) #.sort_index()
#   data=data.set_index(['date'])
#   data=data.set_index(['date']) #.sort_index()
  return code, data


In [3]:
def recalc(code, data):
    if data.size == 0:
        return
    beg_date = data.index[0]
    end_date = data.index[-1]
    today = datetime.datetime.today().date()
    ds = None
    while beg_date <= today:
        if beg_date in data.index:
            ds = data.loc[beg_date]
        else:
            ds.name = beg_date
            data = data.append(ds)
        beg_date = beg_date + datetime.timedelta(1)
    data = data.sort_index()
#     data.to_csv('%s.csv'%code)
    return data


In [6]:
def getStockByDate(stockData, date,code):
    if date < stockData.index[0][0]:
        return None
    try:
        return stockData.loc[date,code]
    except:
        return getStockByDate(stockData, date - datetime.timedelta(1), code)

In [7]:
def getValidDate(year, month, day, nextday = True):
    try:
        return pd.Timestamp(year, month, day)
    except:
        if nextday:
            return getValidDate(year, month + 1, 1)
        else:
            return getValidDate(year, month, day - 1)

In [8]:
getValidDate(2010,11,31)

Timestamp('2010-12-01 00:00:00')

In [9]:
def beforeDate(calcDate, year = 0, month = 0):
    year = year + int(month / 12)
    month = month - int(month / 12) * 12
    if calcDate.month > month:
        result = getValidDate(calcDate.year - year, calcDate.month - month, calcDate.day)
    else:
        result = getValidDate(calcDate.year - year - 1, calcDate.month + 12 - month, calcDate.day)
    
    return result

In [10]:
def afterDate(calcDate, year = 0, month = 0):
    year = year + int(month / 12)
    month = month - int(month / 12) * 12
    if calcDate.month + month > 12:
        result = getValidDate(calcDate.year + year + 1, calcDate.month + month - 12, calcDate.day)
    else:
        result = getValidDate(calcDate.year + year, calcDate.month + month, calcDate.day)
    
    return result


In [11]:
def calcBAYMDateLst(calcDate, dstDate, year, month, before = True):
    out = []
    if before:
        value = eval('beforeDate')(calcDate, year, month)
        out.append(value)
        while value > dstDate:
            calcDate = value
            value = eval('beforeDate')(calcDate, year, month)
            out.append(value)
    else:
        value = eval('afterDate')(calcDate, year, month)
        out.append(value)
        while value < dstDate:
            calcDate = value
            value = eval('afterDate')(calcDate, year, month)
            out.append(value)
    return out

In [12]:
def calcBAYMDate(calcDate, year, month, before = True):
    if before:
        value = eval('beforeDate')(calcDate, year, month)
    else:
        value = eval('afterDate')(calcDate, year, month)
    return value

In [ ]:
today = datetime.date.today()
dstDate = datetime.date(2021,4,2)
o1=calcBAYMDate(dstDate, 7, 6, False)
o1
# testDate(today, 'afterDate', 1)

In [ ]:
def GannCheckOld(dataIn, posB0Day, gannBase = {'y':30,'m':0}, gannList = [{'y':7,'m':6},{'y':2,'m':6},{'y':0,'m':7},{'y':0,'m':7}]):
#     data=dataIn.reset_index()
#     data=data.set_index(['date'])
#     lstDay = data.index[-1]
    code = data.index[-1][1] ##二维索引用
    lastDay = data.index[-1][0] ##数据最大日期
    x0data = getStockByDate(data, posB0Day, code) #基础点1的数据
    xedata = getStockByDate(data, lastDay, code) #基础点1的数据 
    x1wight = 1 ##权重
    pxedata = 0
    pdelta = 0
    tj0=gannList[0] ##基础计算目标
    ##基础点2
    posB1Day = calcBAYMDate(posB0Day, gannBase['y'], gannBase['m'], False)
    pos0Day = None
    pos1Day = None
    if posB1Day > lastDay:
        tj1=gannList[0]
        base2 = calcBAYMDate(posB1Day, tj1['y'], tj1['m'], True)
        base1 = calcBAYMDate(base2, tj1['y'], tj1['m'], True)
        if base2 < lastDay and base1 < lastDay:
            pos0Day = base1
            pos1Day = base2
            x0data = getStockByDate(data, pos0Day, code)
            x1data=getStockByDate(data, pos1Day, code)
            x1wight = x1data.close / x0data.close
            
            pxedata = getStockByDate(data, calcBAYMDate(lastDay, tj1['y'], tj1['m'], True), code) * x1wight
            pdelta = abs((pxedata.close - xedata.close ) / xedata.close )
            if pdelta > 0.05:
                x1wight = x1data.close / x0data.close
            else:
                x1wight = x1data.close / x0data.close
    else:
        x1data = getStockByDate(data, out1, code)
        x1wight = x1data.close / x0data.close
    
    return {'posB0':posB0Day, 'posB1':posB1Day, 'pos0':pos0Day, 'pos1':pos1Day, 'w':x1wight, 'pdata':pxedata, 'pd':pdelta}

In [13]:
def GannCheck(dataIn, posB0Day, gannBase = {'y':30,'m':0}, gannList = [{'y':7,'m':6},{'y':2,'m':6},{'y':0,'m':7},{'y':0,'m':1}]):
#     data=dataIn.reset_index()
#     data=data.set_index(['date'])
#     lstDay = data.index[-1]
    code = data.index[-1][1] ##二维索引用
    lastDay = data.index[-1][0] ##数据最大日期
    x0data = getStockByDate(data, posB0Day, code) #基础点1的数据
    xedata = getStockByDate(data, lastDay, code) #基础点1的数据 
    x1wight = 1 ##权重
    pxedata = 0
    pdelta = 0
    tj0=gannList[0] ##基础计算目标
    ##基础点2
    posB1Day = calcBAYMDate(posB0Day, gannBase['y'], gannBase['m'], False)
    pos0Day = None
    pos1Day = None
    if posB1Day > lastDay:
        result = GannSubCheck(data, posB1Day, gannList)
        if result != None:
            result['posB0'] = posB0Day
            result['posB1'] = posB1Day
        return result
    else:
        print('mloop3', base2)
        pos0Day = posB0Day
        pos1Day = posB1Day
        x0data = getStockByDate(data, pos0Day, code)
        x1data=getStockByDate(data, pos1Day, code)
        x1wight = x1data.close / x0data.close

        pxedata = getStockByDate(data, calcBAYMDate(lastDay, gannBase['y'], gannBase['m'], True), code) * x1wight
        pdelta = abs((pxedata.close - xedata.close ) / xedata.close )
    
        return {'posB0':posB0Day, 'posB1':posB1Day, 'pos0':pos0Day, 'pos1':pos1Day, 'w':x1wight, 'pdata':pxedata, 'pd':pdelta}

In [14]:
def GannSubCheck(data, posB1Day, gannList, delta = 0.1):
    code = data.index[-1][1] ##二维索引用
    lastDay = data.index[-1][0] ##数据最大日期
    xedata = getStockByDate(data, lastDay, code) #基础点1的数据

    tj1=gannList[0]
    pos0Day = None
    pos1Day = None
    x1wight = 1 ##权重
    pxedata = 0
    pdelta = 0
    base2 = calcBAYMDate(posB1Day, tj1['y'], tj1['m'], True)
    base1 = calcBAYMDate(base2, tj1['y'], tj1['m'], True)
    if base2 < lastDay and base1 < lastDay:
        pos0Day = base1
        pos1Day = base2
        x0data = getStockByDate(data, pos0Day, code)
        x1data=getStockByDate(data, pos1Day, code)
        x1wight = x1data.close / x0data.close

        pxedata = getStockByDate(data, calcBAYMDate(lastDay, tj1['y'], tj1['m'], True), code) * x1wight
        pdelta = abs((pxedata.close - xedata.close ) / xedata.close )
        if pdelta > delta:
            if len(gannList) <= 1:
#                 print('None')
                return None
            else:
#                 print('loop1', gannList[1:], posB1Day, pdelta)
                return GannSubCheck(data, posB1Day, gannList[1:], delta)
        else:
#             print('loop2', gannList, posB1Day, pdelta)
            return {'pos0':pos0Day, 'pos1':pos1Day, 'w':x1wight, 'tj1':tj1}
    else:
#         print('loop3', base2)
        return GannSubCheck(data, base2, gannList, delta)

In [16]:
m=MongoIo()
# data=m.get_index_day('000001', st_end = '2022-09-01', st_start = '1990-05-28')
data=m.get_stock_day('600827', st_end = '2022-09-01', st_start = '1990-05-28')
data.tail()

,,open,high,low,close,volume,amount
date,code,,,,,,
2021-03-31,600827,17.75,17.96,17.62,17.78,111591.0,198215968.0
2021-04-01,600827,17.74,18.46,17.73,18.38,193919.0,353269632.0
2021-04-02,600827,18.22,18.48,18.10,18.28,151472.0,276234592.0
2021-04-06,600827,18.20,18.58,17.72,17.89,180264.0,324674080.0
2021-04-07,600827,17.80,18.49,17.71,18.45,203201.0,368431136.0


In [17]:
outs=GannCheck(data, pd.Timestamp(1997,5,6), gannBase = {'y':30,'m':0}, gannList = [{'y':7,'m':6},{'y':2,'m':6},{'y':0,'m':7},{'y':0,'m':1}])
outs

{'pos0': Timestamp('2021-03-06 00:00:00'),
 'pos1': Timestamp('2021-04-06 00:00:00'),
 'w': 1.0975460122699385,
 'tj1': {'y': 0, 'm': 1},
 'posB0': Timestamp('1997-05-06 00:00:00'),
 'posB1': Timestamp('2027-05-06 00:00:00')}

In [18]:
# m=MongoIo()
# # data=m.get_index_day('000001', st_end = '2022-09-01', st_start = '1990-05-28')
# data=m.get_stock_day('600827', st_end = '2022-09-01', st_start = '1990-05-28')
# outs=GannCheck(data, pd.Timestamp(1992,5,26), gannBase = {'y':30,'m':0}, gannList = [{'y':7,'m':6},{'y':2,'m':6},{'y':0,'m':7},{'y':0,'m':1}])
# outs=GannCheck(data, pd.Timestamp(1997,5,6), gannBase = {'y':30,'m':0}, gannList = [{'y':7,'m':6},{'y':2,'m':6},{'y':0,'m':7},{'y':0,'m':1}])
# outs
data['ycz'] = 0
x1wight=outs['w']
# print(x1wight)
gannBase=outs['tj1']
# print(gannBase)
lastDay = data.index[-1][0]
lastDay
code='600827'
for rday in range(-20, 20):
    ycday = lastDay + datetime.timedelta(rday)
#     print(ycday)
    preN = getStockByDate(data, calcBAYMDate(ycday, gannBase['y'], gannBase['m'], True), code) * x1wight
    preD = afterDate(preN.name[0], year=0, month = 7)
    try:
        rclose = data.at[(ycday,code),'close']
#         print(ycday)
        rclose = getStockByDate(data, ycday, code).close
        diff = preN.close - rclose
    except Exception as e:
#         print(preN.name[0])
        rclose = 0.0
        diff = 0
#     print(str(preD)[:10], int(preN.close), rclose, int(diff))
    if rclose == 0 and ycday < lastDay:
#         print(ycday)
        continue
    else:
        if ycday.weekday() > 4:
            continue
        rclose = getStockByDate(data, ycday, code).close
#     print(ycday.weekday())
    data.at[(ycday,code),'ycz']  = preN.close
    data.at[(ycday,code),'close']  = rclose
data.tail(30)

,,open,high,low,close,volume,amount,ycz
date,code,,,,,,,
2021-03-15,600827,16.49,18.04,16.47,18.04,307595.0,545724864.0,0.000000
2021-03-16,600827,18.30,18.55,17.75,18.35,425092.0,779496832.0,0.000000
2021-03-17,600827,18.10,18.66,18.03,18.60,237596.0,437052064.0,0.000000
2021-03-18,600827,18.40,19.10,18.36,18.57,290061.0,543996480.0,20.000000
2021-03-19,600827,18.29,18.82,18.14,18.63,248983.0,461459392.0,20.000000
2021-03-22,600827,18.78,18.88,17.70,18.11,322062.0,587211072.0,20.000000
2021-03-23,600827,18.25,18.29,16.93,17.27,303629.0,533068704.0,19.000000
2021-03-24,600827,17.50,17.62,16.54,16.57,253426.0,430812800.0,19.000000
2021-03-25,600827,16.66,18.18,16.29,17.25,315514.0,546835008.0,18.000000


In [ ]:
xlabel = []
for x in data.iloc[-24:].index:
    xlabel.append(str(x[0])[5:10])
data.iloc[-24:].close.plot(x=xlabel,figsize=(400,500))
data.iloc[-24:].ycz.plot(x=xlabel,figsize=(400,500))


In [ ]:
data.at[(data.index[-1][0],'000001'),'ycz']

In [ ]:
# data.at[[data.index[-1][0],code],'ycz'] = 123
data.at[data.index[-1],'ycz']

In [ ]:
data.index[-1]

In [ ]:
# print(data.iloc[-60])
np.array(data.iloc[-60:].index.levels[0])

In [ ]:
# data3=data.iloc[:-50]
# print(data3.index[-1])
outs4 = []
for x in data.iloc[-30:].index:
    outs4.append(data.loc[x].close)

#     print(data.loc[x].close)
# outs = GannSubCheck(data3, pd.Timestamp(2022,10,16), gannList = [{'y':7,'m':6},{'y':2,'m':6},{'y':0,'m':7},{'y':0,'m':1}])
# outs['prelist']
# outs1 = GannSubCheck(data3, pd.Timestamp(2022,4,26), gannList = [{'y':7,'m':6},{'y':2,'m':6},{'y':0,'m':7},{'y':0,'m':1}])
# outs1['prelist']
print(len(out4))
x = range(-10,20)
print(x)
# plt.plot(x, outs)
plt.title("Matplotlib demo") 
plt.xlabel("x axis caption") 
plt.ylabel("y axis caption") 
plt.plot(x,outs['prelist']) 
plt.plot(x,outs1['prelist']) 
plt.plot(x,outs4) 
# plt.plot(np.array(data.iloc[-60:].index.levels[0]),np.array(data.iloc[-60:].close))
plt.show()

In [ ]:
data.iloc[-60:].close.plot()
data.iloc[-60:].high.plot()

In [ ]:
getStockByDate(data, pd.Timestamp(2013,10,2),'000001')

In [ ]:
def calcBADDate(calcDate, dstDate, day, before = True):
    out = []
    value = None
    if before:
        while value == None or value > dstDate:
            value = calcDate - datetime.timedelta(day)
            out.append(value)
            calcDate = value
    else:
        while value == None or value < dstDate:
            value = calcDate + datetime.timedelta(day)
            out.append(value)
            calcDate = value
    return out    

In [ ]:
calcBADDate(dstDate, today, 50, False)
calcBADDate(today, dstDate, 20, True)


In [ ]:
calcGannDate(dstDate, today, 30, False)

In [ ]:
def calcGannDate(calcDate, dstDate, base = 360, before = True):
    mdays = abs((calcDate - dstDate).days)
    
    intValue = True
    if not isinstance (base, int):
        intValue = False
    yValue = []
    yLabel = 'y' + str(base)
    sValue = []
    sLabel = 's' + str(base)
    mValue = []
    mLabel = 'm' + str(base)
    wValue = []
    wLabel = 'w' + str(base)
    dValue = []
    dLabel = 'd' + str(base)
    if mdays > base * 365:
        ## 年
        if isinstance (base, int):
            year = base
            month = 0
        else:
            year = int(base)
            month = int((base * 10 - int(base) * 10) * 12 / 10)
        yValueT = calcBAYMDate(calcDate, dstDate, year, month, before)
    if intValue and mdays > base * 90:
        # 季
        year = int(base / 3)
        month = (base - 3 * int(base/3)) * 3
        sValue = calcBAYMDate(calcDate, dstDate, year, month, before)
    if intValue and mdays > base * 30:
        # 月
        year = int(base / 12)
        month = base - 12 * int(base / 12) 
        mValue = calcBAYMDate(calcDate, dstDate, year, month, before)
    if intValue and mdays > base * 7:
        # 周
        wValue = calcBADDate(calcDate, dstDate, base * 7, before)
    if intValue and mdays > base:
        # 日
        dValue = calcBADDate(calcDate, dstDate, base, before)
    return {yLabel: yValue, sLabel: sValue, mLabel: mValue, wLabel: wValue, dLabel: dValue}

In [ ]:
datetime.datetime.strptime('2012-12-12','%Y-%m-%d').date()

In [ ]:
def gannCheck(dates, before = True, bday = 20, aday = 10):
    today = datetime.date.today()
    odate = []
    for sdate in dates:
        cdate = datetime.datetime.strptime(sdate,'%Y%m%d').date()
#         print(cdate, today)
        if before:
            outs = gannDateAll(today, cdate, before)
        else:
            outs = gannDateAll(cdate, today, before)
        
#         return outs
        for out in outs:
#             print(out.keys)
            for d in out:
                gdate = out[d]
#                 print(gdate)
                if gdate == None:
                    continue
                else:
                    lout = []
                    for ldate in gdate:
                        cdays = abs((ldate - today).days)
                        if ldate > today:
                            if cdays < aday:
                                lout.append(ldate)
                        else:
                            if cdays < bday:
                                lout.append(ldate)
#                         if abs((ldate - today).days) < 30:
#                             lout.append(ldate)
                    if len(lout) > 0:
                        odate.append({d:lout})
#             return out
    return odate

In [ ]:
def gannDateAll(calcDate, fstDate, before = True):
    result = []
    result.append(calcGannDate(calcDate, fstDate, 360, before))
    result.append(calcGannDate(calcDate, fstDate, 180, before))
    result.append(calcGannDate(calcDate, fstDate, 120, before))
    result.append(calcGannDate(calcDate, fstDate, 90, before))
    result.append(calcGannDate(calcDate, fstDate, 60, before))
    result.append(calcGannDate(calcDate, fstDate, 30, before))
    result.append(calcGannDate(calcDate, fstDate, 28, before))
    result.append( calcGannDate(calcDate, fstDate, 15, before))
    result.append(calcGannDate(calcDate, fstDate, 10, before))
    result.append(calcGannDate(calcDate, fstDate, 7, before))
    result.append(calcGannDate(calcDate, fstDate, 2.5, before))
#     return (y360, y180, y120, y90, y60, y30, y28, y15, y10, y7, y2d5)
#     return result
    return result

In [ ]:
datetime.date(2020,10,31)

In [ ]:
# data.head()?
data.loc[data.high.idxmax()]

In [ ]:
# 1993-02-15
a = gannCheck(['19930601'], False, aday = 30, bday = 10)
a = gannCheck(['19930216'], False, aday = 30, bday = 10)
# a = gannCheck(['19940729'], False, aday = 30, bday = 10)
# 1994-07-29
a

In [ ]:
for b in a:
    print(a[b])

In [ ]:
beforeDate(datetime.date(2021,4,2), 7, 6)

In [ ]:
def jeDateCalc(calcDate, fstDate, base = 360):
    mdays = (calcDate - fstDate).days
    intValue = True
    if not isinstance (base, int):
        intValue = False
    yValue = None
    yLabel = 'y' + str(base)
    sValue = None
    sLabel = 's' + str(base)
    mValue = None
    mLabel = 'm' + str(base)
    wValue = None
    wLabel = 'w' + str(base)
    dValue = None
    dLabel = 'd' + str(base)
    if mdays > base * 365:
        ## 年
        if isinstance (base, int):
            year = base
            month = 0
        else:
            year = int(base)
            month = int((base * 10 - int(base) * 10) * 12 / 10)
        yValue = beforeDate(calcDate, year, month)
    if intValue and mdays > base * 90:
        # 季
        year = int(base / 3)
        month = (base - 3 * int(base/3)) * 3
        sValue = beforeDate(calcDate, year, month)
    if intValue and mdays > base * 30:
        # 月
        year = int(base / 12)
        month = base - 12 * int(base / 12) 
        mValue = beforeDate(calcDate, year, month)
    if intValue and mdays > base * 7:
        # 周
        wValue = calcDate - datetime.timedelta(base * 7)
    if intValue and mdays > base:
        # 日
        dValue = calcDate - datetime.timedelta(base)
    return {yLabel: yValue, sLabel: sValue, mLabel: mValue, wLabel: wValue, dLabel: dValue}

In [ ]:
def jeDateAll(calcDate, fstDate):
    y360 = jeDateCalc(calcDate, fstDate, 360)
    y180 = jeDateCalc(calcDate, fstDate, 180)
    y120 = jeDateCalc(calcDate, fstDate, 120)
    y90 = jeDateCalc(calcDate, fstDate, 90)
    y60 = jeDateCalc(calcDate, fstDate, 60)
    y30 = jeDateCalc(calcDate, fstDate, 30)
    y28 = jeDateCalc(calcDate, fstDate, 28)
    y15 = jeDateCalc(calcDate, fstDate, 15)
    y10 = jeDateCalc(calcDate, fstDate, 10)
    y7 = jeDateCalc(calcDate, fstDate, 7)
    y2d5 = jeDateCalc(calcDate, fstDate, 2.5)
    return (y360, y180, y120, y90, y60, y30, y28, y15, y10, y7, y2d5)

In [ ]:
(code, data) = readTdxLdayFile()
data2=recalc(code, data)
data3 = data2.copy()
data3[1:3]

In [ ]:
jedate=jeDateAll(cdata, bdata)
jedate

In [ ]:
cdata = datetime.date(2021,2,18)
bdata = datetime.date(1990,2,18) 
df = cdata  - bdata
df

In [ ]:
jedate[0]['d360']

In [ ]:
calcDate = jedate[0]['d360']
calcDate

In [ ]:
def calcJePos(calcDate, befDay = 3, aftDay = 3):
    bnum = befDay
    enum = aftDay

    beg=0
    end=0
    mhigh=0
    mlow=0
    hday=0
    lday=0
    graph=''
    while beg == mhigh or beg == mlow or end == mhigh or end == mlow:
        bnum += 1
        enum += 1
        bd=calcDate - datetime.timedelta(bnum)
        ed=calcDate + datetime.timedelta(enum)

        df3=data3.loc[bd:ed]
        beg=df3.index[0]
        end=df3.index[-1]
        mhigh=df3.iloc[df3.close.argmax()].name
        mlow=df3.iloc[df3.close.argmin()].name
        hday = (calcDate - mhigh).days
        lday = (calcDate - mlow).days
        
        graph = calcGraph(calcDate, mhigh, mlow)
        
    return {'high':mhigh, 'low':mlow, 'befDay':bnum, 'aftDay':enum, 'hday':hday, 'lday':lday, 'graph':graph}


In [ ]:
def calcGraph(calcDate, mhigh, mlow):        
    graph = ''
    if mhigh < mlow:## H-L
        if mhigh < calcDate and mlow > calcDate:
            graph = 'HCL'
        else:
            if calcDate > mlow:
                grapho = 'HLC'
            else:
                graph = 'CHL'
    else: ## L-H
        if mlow < calcDate and mhigh > calcDate:
            graph = 'LCH'
        else:
            if calcDate > mhigh:
                grapho = 'LHC'
            else:
                graph = 'CLH'
    return graph

In [ ]:
def calcBefDay(calcDate, befDay = 10):
    df3=data3[calcDate - datetime.timedelta(befDay):calcDate]
    mhigh=df3.iloc[df3.close.argmax()].name
    mlow=df3.iloc[df3.close.argmin()].name
    hday = (calcDate - mhigh).days
    lday = (calcDate - mlow).days
    graph = calcGraph(calcDate, mhigh, mlow)
    print(mhigh, mlow, calcDate)
    if hday == befDay or lday == befDay:
        befDay += 1
        return calcBefDay(calcDate, befDay)
    else:
        return {'hday':hday, 'lday':lday, 'day':befDay, 'graph':graph}
    

In [ ]:
# print(calcJePos(calcDate, befDay = 10, aftDay = 10))
print(calcDate)
print(calcJePos(calcDate))

In [ ]:
print(calcBefDay(calcDate))

In [ ]:
bnum = 0
enum = 0

beg=0
end=0
mhigh=0
mlow=0

while beg == mhigh or beg == mlow or end == mhigh or end == mlow:
    bnum += 1
    enum += 1
    bd=calcDate - datetime.timedelta(bnum)
    ed=calcDate + datetime.timedelta(enum)

    df3=data3.loc[bd:ed]
    beg=df3.index[0]
    end=df3.index[-1]
    mhigh=df3.iloc[df3.close.argmax()].name
    mlow=df3.iloc[df3.close.argmin()].name


In [ ]:
print(data3.iloc[data3.close.argmax()].name)
print(data3.iloc[data3.close.argmin()].name)

In [ ]:
def calcGannPos(data, year = 0, month = 0, days = 0):
    if step == 0:
        hdate = data.iloc[data3.close.argmax()].name
        hvalue = data.loc[hdate]
        ldate = data.iloc[data3.close.argmin()].name
        lvalue = data.loc[ldate]
        return {'h': hvalue, 'l': lvalue}
    else:
#         if type = 'y':
        beg = data.index[0]
        ndate = afterDate(beg, step, 0)
        end = data.index[-1]
        outa=[]
        while ndate < end:
            cdata = data.loc[beg:ndate]
            hdate = cdata.iloc[cdata.close.argmax()].name
            ldate = cdata.iloc[cdata.close.argmin()].name
            t1 = {'h': data.loc[hdate], 'l': data.loc[ldate]}
            outa.append(t1)
            beg = ndate + datetime.timedelta(1)
            ndate = afterDate(beg, step, 0)
            
        return outa
        

In [ ]:
calcGannPos(data3, step = 1)

In [ ]:
print('h',mhigh)
print('l', mlow)
print('b',beg)
print('e',end)
print(calcDate)

In [ ]:
# df3

In [ ]:
data3.loc[calc_date]

In [ ]:
calc_date

In [ ]:
calc_date = datetime.date.today()

In [ ]:
df=data.loc[beg_date]
df

In [ ]:
df.name=datetime.date(1990, 12, 20)
df

In [ ]:
dfn = pd.DataFrame()

In [ ]:
dfn=dfn.append(df)

dfn

In [ ]:
type(dfn)

In [ ]:
today = datetime.datetime.today().date()
data.loc[beg_date]

In [ ]:
df1=data.loc[beg_date]

In [ ]:
end_date = datetime.date(1990,12,31)
end_date

In [ ]:
d2=data[0:9].copy()
d2

In [ ]:
# d2.at[datetime.date(1990,12,22),'code']='sh111'
ndf = d2.loc[datetime.date(1990,12,21)]
# d2.append(datetime.date(1990,12,22))
ndf['name']='1234'

In [ ]:
ndf

In [ ]:
ndf.index['name']=datetime.date(1990,12,22)

In [ ]:
data=data.set_index(['tradeDate'])

In [ ]:
df=datetime.datetime.strptime(str(data.index[0]),'%Y%m%d').date()

In [ ]:
from easyquant import MongoIo
from easyquant.indicator.base import *

In [ ]:
m=MongoIo()
data=m.get_index_day('000001', st_end = '2022-09-01', st_start = '1993-05-28')

In [ ]:
data.loc[data.close.idxmin()]

In [ ]:
VOL = data.volume
CLOSE = data.close
C = data.close
H = data.high
L = data.low
O = data.open
VAR2 = CLOSE * VOL
VAR3 = EMA((EMA(VAR2, 3) / EMA(VOL, 3) + EMA(VAR2, 6) / EMA(VOL, 6)
            + EMA(VAR2, 12) / EMA(VOL, 12) + EMA(VAR2, 24) / EMA(VOL, 24)) / 4, 13)
白线 = 1.06 * VAR3
MA4 = MA(C, 4)
MA24 = MA(C, 24)
# C1 = C >= MA4
# C2 = C < MA4
#
# # IF(MA4 >= REF(MA4, 1), MA4, DRAWNULL), COLORRED, LINETHICK2;
JJ = (3 * C + H + L + O) / 6
VAR1 = (8 * JJ + 7 * REF(JJ, 1) + 6 * REF(JJ, 2) + 5 * REF(JJ, 3) + 4 * REF(JJ, 4)
        + 3 * REF(JJ, 5) + 2 * REF(JJ, 6) + REF(JJ, 8)) / 36
TJ2 = IFAND4(VOL == HHV(VOL, 10), VOL > 2 * REF(VOL, 1), CLOSE > VAR1, C > REF(C, 1), True, False)
# LJL = FILTER(TJ2, 5)
TJ1 = (JJ-白线) /白线 * 100

HVOL = HHV(VOL, 34)
HNUM = BARSLAST(HVOL == VOL)
HH = REF(H, HNUM)
HL = REF(L, HNUM)
JZBD = IFAND4(HNUM > 10, C > HL, H > HH, C < HH * 1.06, True, False)
JZBD_S = IFAND3(HNUM > 10, C > HL, H > HH, True, False)


In [ ]:
data.close.argmax()

In [ ]:
HNUM[90:]
# HH[185:]
# C[185:]

In [ ]:
BARSLAST(HH == VOL)

In [ ]:
def tdx_bjmm_jzmd(data):
    # AMOUNT = data.amount
    VOL = data.volume
    CLOSE = data.close
    C = data.close
    H = data.high
    L = data.low
    O = data.open
    VAR2 = CLOSE * VOL
    VAR3 = EMA((EMA(VAR2, 3) / EMA(VOL, 3) + EMA(VAR2, 6) / EMA(VOL, 6)
                + EMA(VAR2, 12) / EMA(VOL, 12) + EMA(VAR2, 24) / EMA(VOL, 24)) / 4, 13)
    白线 = 1.06 * VAR3
    MA4 = MA(C, 4)
    MA24 = MA(C, 24)
    # C1 = C >= MA4
    # C2 = C < MA4
    #
    # # IF(MA4 >= REF(MA4, 1), MA4, DRAWNULL), COLORRED, LINETHICK2;
    JJ = (3 * C + H + L + O) / 6
    VAR1 = (8 * JJ + 7 * REF(JJ, 1) + 6 * REF(JJ, 2) + 5 * REF(JJ, 3) + 4 * REF(JJ, 4)
            + 3 * REF(JJ, 5) + 2 * REF(JJ, 6) + REF(JJ, 8)) / 36
    TJ2 = IFAND4(VOL == HHV(VOL, 10), VOL > 2 * REF(VOL, 1), CLOSE > VAR1, C > REF(C, 1), True, False)
    # LJL = FILTER(TJ2, 5)
    TJ1 = (JJ-白线) /白线 * 100

    HVOL = HHV(VOL, 34)
    HNUM = BARSLAST(HVOL == VOL)
    HH = REF(H, HNUM)
    HL = REF(L, HNUM)
    JZBD = IFAND4(HNUM > 10, C > HL, H > HH, C < HH * 1.06, True, False)
    JZBD_S = IFAND3(HNUM > 10, C > HL, H > HH, True, False)
    # # MACDTJ
    # SHORT =12
    # LONG = 26
    # MID = 9
    # EMASHORT = EMA(CLOSE, SHORT)
    # EMALONG = EMA(CLOSE, LONG)
    # DIF = EMASHORT - EMALONG
    # DEA = EMA(DIF, MID)
    # MACD = (DIF-DEA) * 2
    # # MACDTJ = IFAND4(MACD>0, DIF > 0, DEA > 0, CROSS(DIF, DEA), True, False)
    # MACDTJ = IFAND3(MACD > 0, DIF > 0, DEA > 0, True, False)

    # B_1 = IFAND3(TJ1 >= 0, REF(TJ1,1) < 0, MACDTJ, 1, 0)
    # B_1 = IFAND(TJ1 >= 0, REF(TJ1, 1) < 0, 1, 0)
    # B_1 = IFAND4(TJ1 > 0, REF(TJ1, 1) < 0, TJ2, MACDTJ, 1, 0)
    # B_1 = IFAND3(TJ1 > 0, REF(TJ1, 1) < 0, TJ2, 1, 0)
    B_1 = IFAND4(TJ1 > 0, REF(TJ1, 1) < 0, TJ2, JZBD, 1, 0)
    S_1 = IFAND(TJ1 < 0, REF(TJ1, 1) > 0, 1, -1)
    # S_1 = IFAND(C < MA24, REF(C,1) > REF(MA24,1), 1, -1)
    # return B_1, B_1, False
    # return B_1, -1, False
    return B_1, S_1, False


In [ ]:
H3=tdx_bjmm_jzmd(data)

In [ ]:
a,b,c = H3

In [ ]:
b.tail()

In [ ]:
import QUANTAXIS as QA
import pyecharts.options as opts
from pyecharts.charts import Candlestick
from pyecharts.globals import CurrentConfig, NotebookType
CurrentConfig.NOTEBOOK_TYPE = NotebookType.JUPYTER_LAB

code=["000735"]
start_date="2020-01-01"
end_date="2021-01-01"

ax = QA.QA_fetch_stock_day_adv(code,start_date, end_date).data.reset_index(1)
# data = QA.QA_fetch_stock_day_adv(['000001', '000002', '000004', '600000'], '2017-09-01', '2018-05-20')

x_data = ax.index.map(str).values.tolist()
y_data = ax.loc[:,['open',  'close','low','high']].values.tolist()

r =(
    Candlestick(init_opts=opts.InitOpts(width="1440px", height="720px"))
    .add_xaxis(xaxis_data=x_data)
    .add_yaxis(series_name="", y_axis=y_data)
    .set_series_opts()
    .set_global_opts(
        xaxis_opts=opts.AxisOpts(is_scale=True),
        title_opts=opts.TitleOpts(title=code),
    )
)
r.load_javascript()
r.render_notebook()


In [ ]:
stamp=pd.Timestamp(2021,4,8)

In [ ]:
str(stamp)[:10]

In [ ]:
time.strptime(stamp, '%Y')